In [1]:
using DifferentialEquations
using Plots
#using Plotly
using Interact
using Pkg
#Pkg.add("WebIO")
#using WebIO
#WebIO.install_jupyter_nbextension()

#plotly()

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-398754708919860767\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-17777404504097613520\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-398754708919860767\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

In [2]:
using DifferentialEquations
using Plots
using Plotly
using Statistics


mutable struct SIRS_Game
    NS::Int             #number of strategies
    XS::Vector{Float64} #strategy state
    XF::Vector{Float64} #fatigue state
    #epidemics state
    S::Float64
    I::Float64
    R::Float64
    #parameters of the epidemics
    β #this can be a function
    γ::Float64
    α::Float64
    #payoff function
    F
    #parameters of the payoff
    ζ::Vector{Float64} 
    ϑ::Vector{Float64} 
    θ::Vector{Float64} 
    ν::Vector{Float64} 
    ξ::Vector{Float64} 
    function SIRS_Game(ns::Int, f, b, g ,a) 
        F  = f
        NS = ns
        β  = b 
        γ  = g
        α  = a
        XS = zeros(NS)
        XF = zeros(NS)
        ζ  = zeros(NS)
        ϑ  = zeros(NS)
        θ  = zeros(NS)
        ν  = zeros(NS) 
        ξ  = zeros(NS)
        new(NS,XS,XF,1.0,0.0,0.0,β,γ,α,F,ζ,ϑ,θ,ν,ξ)
    end
end



function r(strategy,index::Int)
    5*(strategy-1)+index
end

function r(strategy,index::String)
    if(index=="S" || index=="s")  
        5*(strategy-1)+1
    end
    if(index=="I" || index=="i")  
        5*(strategy-1)+2
    end
    if(index=="R" || index=="r")  
        5*(strategy-1)+3
    end
    if(index=="X" || index=="x")  
        5*(strategy-1)+4
    end
    if(index=="F" || index=="f")  
        5*(strategy-1)+5
    end
end


# population dynamics (~revision protocol) 
function smith(g::SIRS_Game,x::Vector{Float64},t::Float64,i, F)
    sum = 0
    for j ∈ 1:g.NS
        sum +=  x[r(j,4)]*max(F(g,x,t,i)-F(g,x,t,j),0)
    end
    for j ∈ 1:g.NS
        sum += -x[r(i,4)]*max(F(g,x,t,j)-F(g,x,t,i),0)
    end
    
    sum
end


# payoff for the different strategies
function fp(g,x,t,j)
    if j ∈ 1:g.NS
        I_dot = g.β[j]*x[r(j,1)]*x[r(j,2)]-g.γ*x[r(j,2)] #for now, replace with the I_dot term
        -1*(g.ζ[j]+g.ϑ[j]*I_dot+g.θ[j]*x[r(j,2)]+g.ν[j]*x[r(j,5)]+g.ξ[j]*x[r(j,4)])
    end
end

# total dynamics
function h!(du,u,p,t)
   
    #dynamic per strategy
    for i ∈ 1:p.NS
        # epidemic dynamic
        du[r(i,1)] = -u[r(i,1)]*sum([p.β[j]*u[r(j,4)]*u[r(j,2)] for j ∈ 1:p.NS]) + p.α*u[r(i,3)]
        du[r(i,2)] =  u[r(i,1)]*sum([p.β[j]*u[r(j,4)]*u[r(j,2)] for j ∈ 1:p.NS]) - p.γ*u[r(i,2)]
        du[r(i,3)] =  p.γ*u[r(i,2)] - p.α*u[r(i,3)]
        # game dynamic
        du[r(i,4)] = smith(p,u,t,i,p.F)
        # fatigue dynamics
        du[r(i,5)] = 0.03*(u[r(i,4)] - u[r(i,5)])
    end

end
   
a = 0.07
g0 = SIRS_Game(2,fp,[4.0*a,(a/2)],a ,a/2)

I = 0.0001
R = 0.0001
S = 1.0-I-R

W = [S;0.0001;R;1.0;0.0;1.0;0.0;0.0;0.0;0.0]
##

g0.ζ .= [8.0,6.0] 
g0.ϑ .= [10.0,2.0]
g0.θ .= [1.0,9.0] 
g0.ν .= [0.5,3.0] 
g0.ξ .= [1.0,1.0] 

#=
best_g0 = g0
best_std = 0
for j=1:1



    prob = ODEProblem(h!,[SIR;XS;XF],[0.0,4*360.0],g0)
    sol = solve(prob)
    if( std(sol(800:0.1:900)[2,:]) > best_std )
        best_g0 = g0
        best_std = std(sol(800:0.1:900)[2,:])
    end
end

prob = ODEProblem(h!,[SIR;XS;XF],[0.0,2*360.0],best_g0)
=#



2-element Vector{Float64}:
 1.0
 1.0

In [10]:
my_range = 0.0:0.01:10.0
@manipulate for a =0.01:0.01:0.50, gg=0.1:0.1:10, ζ1=my_range, ζ2=my_range, ϑ3=100 .*my_range,
    ϑ4=100 .*my_range, θ5=my_range, θ6=my_range, ν7=my_range, ν8=my_range, ξ9=my_range, ξ10=my_range
    g0 = SIRS_Game(2,fp,[gg*a,(a/2)],a ,a/2)

    I = 0.01
    R = 0.01
    S = 1.0-I-R

    W = [S;I;R;1.0;0.0;1.0;0.0;0.0;0.0;0.0]
    ##
    g0.ζ .= [ζ1,ζ2] 
    g0.ϑ .= [ϑ3,ϑ4]
    g0.θ .= [θ5,θ6] 
    g0.ν .= [ν7,ν8] 
    g0.ξ .= [ξ9,ξ10] 

    prob = ODEProblem(h!,W ,[0.0,420.0],g0)
    sol = solve(prob,Tsit5())
    #plot(sol, label=["S" "I" "R" "s1" "s2" "f1" "f2"])
    Plots.plot(sol, vars=r.([1,2]',2), label=false)
    #xlabel!("Time (day)")
    #ylabel!("Infected Population (ratio)")
end  


Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["a"], Dict{Symbol, Any}(:className => "interact ", :style => Dict{Any, Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol, Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol, Any}(:max => 50, :min => 1, :attributes => Dict{Any, Any}(:type => "range", Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}", "orient" => "horizontal"), :step => 1, :className => "slider slider is-fullwidth", :style => Dict{Any, Any}()))], Dict{Symbol, Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol, Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol, Any}(:className => "interact-flex-row-right"))], Dict{Symbol, Any}(:className => "interact-flex-row interact-widget")), Dict{String, Tuple{Observables.AbstractObservable, Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing), "index" => (Observable{Any} with 2 listeners. Value:
25, nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "/home/jair/.julia/packages/Knockout/IP1uR/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/home/jair/.julia/packages/Knockout/IP1uR/src/../assets/knockout_punches.js"), Asset("js", nothing, "/home/jair/.julia/packages/InteractBase/Zfu5P/src/../assets/all.js"), Asset("css", nothing, "/home/jair/.julia/packages/InteractBase/Zfu5P/src/../assets/style.css"), Asset("css", nothing, "/home/jair/.julia/packages/Interact/SbgIk/src/../assets/bulma_confined.min.css")], Dict{Any, Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")], "index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(32), Set{AbstractConnection}(), Condition(Base.InvasiveLinkedList{Task}(Task (runnable) @0x00007fd3dcf41660, Task (runnable) @0x00007fd3dcf41660), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"0.01\",\"0.02\",\"0.03\",\"0.04\",\"0.05\",\"0.06\",\"0.07\",\"0.08\",\"0.09\",\"0.1\",\"0.11\",\"0.12\",\"0.13\",\"0.14\",\"0.15\",\"0.16\",\"0.17\",\"0.18\",\"0.19\",\"0.2\",\"0.21\",\"0.22\",\"0.23\",\"0.24\",\"0.25\",\"0.26\",\"0.27\",\"0.28\",\"0.29\",\"0.3\",\"0.31\",\"0.32\",\"

In [4]:

##
png("infected")
##

plot(sol, vars=[1,2,3], label=["S" "I" "R"])
xlabel!("Time (day)")
ylabel!("Population (ratio)")

##
png("sir")
##

plot(sol, vars=[4,5,6,7], label=["s1" "s2" "f1" "f2"])
xlabel!("Time (day)")
ylabel!("Population (ratio)")

##
png("strategies_fatigue")
##

LoadError: UndefVarError: plot not defined